In [495]:
import pandas as pd
import numpy as np
import math
import datetime

In [597]:
#---------------------------------------------------------#
# Prototyping
#---------------------------------------------------------#

In [549]:
# get the csv file

In [596]:
# file location for the csv file
file = "./noon_data.csv"

# Return all but first 2 lines of csv to get data:
df = pd.read_csv(file)
df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
a=df.iterrows()
i = 0
for index,row in a:
    if math.isnan(row[2]):
        break
    else:
        i = i + 1
table = df[0:i]

In [369]:
# define dictionary for new table

In [561]:
eds_cols = {}
window = 7

In [562]:
# sort date based on time window

In [563]:
col_name = 'Date'
counter = 0
date_string = ''
new_col = {col_name:[]}

In [564]:
for x in df[col_name]:
    if counter == 0:
        # store the first date to date_string
        date_string = date_string + x + '-'
        # increase counter
        counter = counter + 1
    elif counter == ((window*2)-1):
        # store the last date
        date_string = date_string + x
        # store to dictionary for dataframe for pre and post entry
        new_col[col_name].append(date_string)
        new_col[col_name].append(date_string)
        # reset date_string
        date_string = ''
        # reset counter
        counter = 0
    else:
        # increase counter
        counter = counter + 1
eds_cols.update(new_col)   

In [565]:
# solar noon time sorting

In [566]:
col_name = 'Time'
counter = 0
index_counter = 0
pre_time_data=[]
post_time_data=[]
new_col = {col_name:[]}

In [567]:
for x in df[col_name]:
    # convert time to seconds
    a,b,c = x.split(':')
    hour = int(a)*60*60
    minute = int(b)*60
    second = int(c)
    total_sec = hour + minute + second
    if counter == ((window*2)-1):
        # append the nth data for post time
        post_time_data.append(total_sec)
        index_counter = 0
        # take average seconds
        pre_time_avg = sum(pre_time_data)/window
        post_time_avg = sum(post_time_data)/window
        # convert seconds back to time string using datetime
        pre_time = str(datetime.timedelta(seconds = int(pre_time_avg)))
        post_time = str(datetime.timedelta(seconds = int(post_time_avg)))
        # append to the dictionary
        new_col[col_name].append(pre_time)
        new_col[col_name].append(post_time)
        # reset counter
        counter = 0
        pre_time_data = []
        post_time_data = []
    else:
        # seperate pre and post time data points
        if index_counter % 2 == 0:
            # append time data for pre measurements
            pre_time_data.append(total_sec)
            index_counter = index_counter + 1
        else:
            # append time data for post measurements
            post_time_data.append(total_sec)
            index_counter = 0
        # increase counter
        counter = counter + 1
eds_cols.update(new_col)        

In [568]:
# numerical data sorting

In [569]:
col_name = 'Temperature(C)'
counter = 0
index_counter = 0
pre_data = []
post_data = []
pre_avg = 0
post_avg = 0
new_col = {col_name:[]}

In [570]:
for x in df[col_name]:
    if counter == ((window*2)-1):
        # seperate pre and post for nth data
        post_data.append(x)
        index_counter=0
        # get average value from pre post lists
        pre_avg = sum(pre_data)/window
        post_avg = sum(post_data)/window
        # append results to new dataframe
        new_col[col_name].append(pre_avg)
        new_col[col_name].append(post_avg)
        # reset counter
        counter = 0
        pre_data=[]
        post_data=[]
    else:
        # seperate pre and post data points
        if index_counter % 2 == 0:
            pre_data.append(x)
            index_counter = index_counter + 1
        else:
            post_data.append(x)
            index_counter = 0
        # increase counter
        counter = counter + 1
eds_cols.update(new_col)

In [571]:
eds_cols

{'Date': ['1/27/2020-2/2/2020',
  '1/27/2020-2/2/2020',
  '2/3/2020-2/9/2020',
  '2/3/2020-2/9/2020'],
 'Time': ['12:34:30', '12:35:55', '12:33:26', '12:34:51'],
 'Temperature(C)': [32.42857142857143,
  32.42857142857143,
  32.714285714285715,
  32.714285714285715]}

In [572]:
# pre/post label sorting, 

In [573]:
col_name = 'PRE/POST'
col_name2 = 'EDS/CTRL(#)'
counter = 0
new_col = {col_name:[]}
new_col2= {col_name2:[]}

In [574]:
for x in df[col_name]:
    if counter == ((window*2)-1):
        # append PRE
        new_col[col_name].append("PRE")
        # append POST
        new_col[col_name].append("POST")
        # append EDS number
        new_col2[col_name2].append("EDS1")
        new_col2[col_name2].append("EDS1")
        # reset counter
        counter = 0
    else:
        # increase counter
        counter = counter + 1
eds_cols.update(new_col)
eds_cols.update(new_col2)

In [575]:
# create new dataframe

In [576]:
eds_df = pd.DataFrame(eds_cols)
eds_df

,Date,Time,Temperature(C),PRE/POST,EDS/CTRL(#)
0,1/27/2020-2/2/2020,12:34:30,32.428571,PRE,EDS1
1,1/27/2020-2/2/2020,12:35:55,32.428571,POST,EDS1
2,2/3/2020-2/9/2020,12:33:26,32.714286,PRE,EDS1
3,2/3/2020-2/9/2020,12:34:51,32.714286,POST,EDS1


In [534]:
#---------------------------------------------------------#
# Transform to functions to use
#---------------------------------------------------------#

In [603]:
def read_data(name):
    # file location for the csv file
    file = name #"./noon_data.csv"
    # return pandas dataframe of the csv file
    df = pd.read_csv(file)
    # remove all NaN entries
    df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
    a=df.iterrows()
    i = 0
    for index,row in a:
        if math.isnan(row[2]):
            break
        else:
            i = i + 1
    table = df[0:i]
    
    return table

In [535]:
def sort_dates(eds_cols, window):
    # declare initial variables
    col_name = 'Date'
    counter = 0
    date_string = ''
    new_col = {col_name:[]}
    # go through the Date column of the noon data csv file
    for x in df[col_name]:
        if counter == 0:
            # store the first date to date_string
            date_string = date_string + x + '-'
            # increase counter
            counter = counter + 1
        elif counter == ((window*2)-1):
            # store the last date
            date_string = date_string + x
            # store to dictionary for dataframe for pre and post entry
            new_col[col_name].append(date_string)
            new_col[col_name].append(date_string)
            # reset date_string
            date_string = ''
            # reset counter
            counter = 0
        else:
            # increase counter
            counter = counter + 1
    eds_cols.update(new_col)
    return eds_cols

In [544]:
def sort_time(eds_cols, window):
    # declare initial variables
    col_name = 'Time'
    counter = 0
    index_counter = 0
    pre_time_data=[]
    post_time_data=[]
    new_col = {col_name:[]}
    # go through the Time column of the noon data csv file
    for x in df[col_name]:
        # convert time to seconds
        a,b,c = x.split(':')
        hour = int(a)*60*60
        minute = int(b)*60
        second = int(c)
        total_sec = hour + minute + second
        if counter == ((window*2)-1):
            # append the nth data for post time
            post_time_data.append(total_sec)
            index_counter = 0
            # take average seconds
            pre_time_avg = sum(pre_time_data)/window
            post_time_avg = sum(post_time_data)/window
            # convert seconds back to time string using datetime
            pre_time = str(datetime.timedelta(seconds = int(pre_time_avg)))
            post_time = str(datetime.timedelta(seconds = int(post_time_avg)))
            # append to the dictionary
            new_col[col_name].append(pre_time)
            new_col[col_name].append(post_time)
            # reset counter
            counter = 0
            pre_time_data = []
            post_time_data = []
        else:
            # seperate pre and post time data points
            if index_counter % 2 == 0:
                # append time data for pre measurements
                pre_time_data.append(total_sec)
                index_counter = index_counter + 1
            else:
                # append time data for post measurements
                post_time_data.append(total_sec)
                index_counter = 0
            # increase counter
            counter = counter + 1
    eds_cols.update(new_col)
    return eds_cols

In [539]:
def sort_labels(eds_cols, window):
    # declare initial variables
    col_name = 'PRE/POST'
    col_name2 = 'EDS/CTRL(#)'
    counter = 0
    new_col = {col_name:[]}
    new_col2= {col_name2:[]}
    # go through the labels column of the noon data csv file
    for x in df[col_name]:
        if counter == ((window*2)-1):
            # append PRE
            new_col[col_name].append("PRE")
            # append POST
            new_col[col_name].append("POST")
            # append EDS number
            new_col2[col_name2].append("EDS1")
            new_col2[col_name2].append("EDS1")
            # reset counter
            counter = 0
        else:
            # increase counter
            counter = counter + 1
    eds_cols.update(new_col)
    eds_cols.update(new_col2)
    return eds_cols

In [583]:
def sort_data(name, eds_cols, window):
    # declare initial variables
    col_name = name #'Temperature(C)'
    counter = 0
    index_counter = 0
    pre_data = []
    post_data = []
    pre_avg = 0
    post_avg = 0
    new_col = {col_name:[]}
    # go through the measurements data columns of the noon data csv file
    for x in df[col_name]:
        if counter == ((window*2)-1):
            # seperate pre and post for nth data
            post_data.append(x)
            index_counter=0
            # get average value from pre post lists
            pre_avg = sum(pre_data)/window
            post_avg = sum(post_data)/window
            # append results to new dataframe
            new_col[col_name].append(pre_avg)
            new_col[col_name].append(post_avg)
            # reset counter
            counter = 0
            pre_data=[]
            post_data=[]
        else:
            # seperate pre and post data points
            if index_counter % 2 == 0:
                pre_data.append(x)
                index_counter = index_counter + 1
            else:
                post_data.append(x)
                index_counter = 0
            # increase counter
            counter = counter + 1
    eds_cols.update(new_col)
    return eds_cols

In [608]:
def get_avg_noon_data(cols_list, window):
    # read the noon_data csv file
    df = read_data("./noon_data.csv")
    # declare new dictionary for avg data
    eds_cols = {}
    # sort the date
    eds_cols = sort_dates(eds_cols, window)
    # sort the time
    eds_cols = sort_time(eds_cols, window)
    # sort the pre/post, EDS number
    eds_cols = sort_labels(eds_cols, window)
    # sort all the numerical data
    for x in cols_list:
        eds_cols = sort_data(x, eds_cols, window)
    # create new dataframe
    eds_df = pd.DataFrame(eds_cols)
    return eds_df

In [616]:
# declare metrics for noon data table
cols_list = ['Temperature(C)', 'Humidity(%)', 'GPOA(W/M2)', 'OCV(V)', 'SCC(A)', 'Power(W)', 'PR', 'SR']
# call the function
eds = get_avg_noon_data(cols_list, 15)
eds

,Date,Time,PRE/POST,EDS/CTRL(#),Temperature(C),Humidity(%),GPOA(W/M2),OCV(V),SCC(A),Power(W),PR,SR
0,1/27/2020-2/10/2020,12:34:01,PRE,EDS1,32.666667,28.160000,276.000000,17.133333,0.414667,30.066667,95.133333,97.133333
1,1/27/2020-2/10/2020,12:35:20,POST,EDS1,32.666667,30.333333,242.666667,17.133333,0.432000,31.000000,95.133333,97.133333
